In [1]:
import numpy as np
import pandas as pd
import sklearn
import tqdm

from sklearn.tree import export_graphviz
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import train_test_split as TTS
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import accuracy_score as accuracy, recall_score as recall,roc_auc_score as auc
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score,recall_score, f1_score,roc_auc_score

import matplotlib.pyplot as plt
import time

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import imblearn
from imblearn.over_sampling import SMOTE

import torch
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import torch.utils.data as Data

from xgboost import XGBClassifier as XGBC
from sklearn.tree import export_graphviz

data = pd.read_csv(r"newdata_coronary_health_step3.csv",error_bad_lines=False)
data = data[['年龄','齿列_sum','平均收缩压','WHtR','心电图label','空腹血糖','症状_sum','体温','血尿素氮','血小板','label','性别','local']]
# data.index = range(data.shape[0])
data.info()


fedata = data[data['gender']==2.0]
fedata.drop(['gender'],axis = 1,inplace = True)
print(fedata.shape)

fedata_train = fedata[fedata['local']!=5.0]
fedata_External = fedata[fedata['local']==5.0]
fedata = fedata_train
fedata.drop(['local',],axis = 1,inplace = True)

feX = fedata.iloc[:,fedata.columns != "label"]
fey = fedata.iloc[:,fedata.columns == "label"]
scaler = StandardScaler() #实例化
fex_train, fex_valid, fey_train, fey_valid = train_test_split(feX,fey,test_size=0.3)
sm = SMOTE(random_state=42) #实例化
fex_train, fey_train = sm.fit_resample(fex_train,fey_train)
fex_valid, fey_valid = sm.fit_resample(fex_valid, fey_valid)
fey_train = fey_train.iloc[:,0].ravel()
fey_valid = fey_valid.iloc[:,0].ravel()
# 数据处理
fex_train = torch.tensor(fex_train.values, dtype = torch.float)
fey_train = torch.tensor(fey_train)
fex_valid = torch.tensor(fex_valid.values, dtype = torch.float)
fey_valid = torch.tensor(fey_valid)
#tensor 通 array
print(fex_train.shape, fey_train.shape)
print(fex_valid.shape, fey_valid.shape)
print("------end-------")

class Batch_Net(nn.Module):

    def __init__(self,in_dim,n_hidden_1,n_hidden_2,out_dim):
        super(Batch_Net,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(in_dim,n_hidden_1),
            nn.BatchNorm1d(n_hidden_1),
            nn.ReLU(True))
        self.layer2 = nn.Sequential(
            nn.Linear(n_hidden_1,n_hidden_2),
            nn.BatchNorm1d(n_hidden_2),
            nn.ReLU(True))
        self.layer3 = nn.Sequential(
            nn.Linear(n_hidden_2,out_dim),
            nn.Softmax())

    def forward(self,x):
        hidden_1_out = self.layer1(x)
        hidden_2_out = self.layer2(hidden_1_out)
        out = self.layer3(hidden_2_out)
        return out


batch_size = 256
learning_rate = 0.001

fetrain_dataset = Data.TensorDataset(fex_train, fey_train)
fetest_dataset = Data.TensorDataset(fex_valid, fey_valid)
fetrain_loader = DataLoader(fetrain_dataset,batch_size=batch_size,shuffle=True,drop_last=True)

print(fetrain_loader)
print(fex_train.size(), fey_train.size())

feBatch_model = Batch_Net(10,7,4,2)  
##########################################
# feBatch_model
##########################################
if torch.cuda.is_available():
    feBatch_model = feBatch_model.cuda()
criterion = nn.CrossEntropyLoss()  #交叉熵损失
optimizer = optim.RMSprop(feBatch_model.parameters(),lr=learning_rate) #优化函数为随机梯度下降
epoch = 0
for fe_data in fetrain_loader:
    xtrain, ytrain = fe_data

    if torch.cuda.is_available():
        xtrain = xtrain.cuda()
        ytrain = ytrain.cuda()
    else:
        xtrain = Variable(xtrain)
        ytrain = Variable(ytrain)
    ytrain=ytrain.long()
    out = feBatch_model(xtrain)
    loss = criterion(out, ytrain)
    print_loss = loss.data.item()
 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    epoch+=1

feBatch_pred = feBatch_model(mx_valid)


large = 28; med = 20; small = 12
params = {'axes.titlesize': med, #子图上的标题字体大小
          'legend.fontsize': 28, #图例的字体大小
          'legend.loc': 'lower right',
          #'figure.figsize': (16, 10), #图像的画布大小
          'axes.labelsize': 28, #标签的字体大小
          'xtick.labelsize': 28, #x轴上的标尺的字体大小
          'ytick.labelsize': 28, #y轴上的标尺的字体大小
          'figure.titlesize': 28} #整个画布的标题字体大小
plt.rcParams.update(params) #设定各种各样的默认属性
# plt.style.use('seaborn-whitegrid') #设定整体风格
# sns.set_style("white") #设定整体背景风格

from sklearn.metrics import roc_curve
from sklearn import metrics

FPR_Batch, Recalll_Batch, thresholds_Batch = roc_curve(fey_valid.tolist(),feBatch_pred[:,1].tolist(),pos_label=1)


from sklearn.metrics import roc_auc_score as AUC
area_Batch = AUC(my_valid.tolist(),feBatch_pred[:,1].tolist())



maxindex_Batch = (Recalll_Batch - FPR_Batch).tolist().index(max(Recalll_Batch - FPR_Batch))
_xgb))

plt.figure(figsize=(10,7), dpi= 500)
bwith = 3
TK = plt.gca()#获取边框
TK.spines['bottom'].set_linewidth(bwith)
TK.spines['left'].set_linewidth(bwith)
TK.spines['top'].set_linewidth(bwith)
TK.spines['right'].set_linewidth(bwith)

plt.plot(FPR_Batch, Recalll_Batch, linestyle='-',color='red',linewidth = 6.0,label='Fully connected layer_ROC curve (area =  0.8671)' % area_Batch)

plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
bwith = 3 

plt.scatter(FPR_Batch[maxindex_Batch],Recalll_Batch[maxindex_Batch],c="red",s=150)



plt.xlim([-0.02,1.02])
plt.ylim([-0.02,1.02])
font2 = {
    'family':'Times New Roman',
    'weight':'normal',
    'size':35,
    'style':'italic'
}
font1 = {
    'family':'Times New Roman',
    'weight':'normal',
    'size':20
}
plt.xlabel('1-Specificity',font2)
plt.ylabel('Sensitivily',font2)
l1 = plt.legend(loc = "lower right",fontsize = 20,prop=font1)
plt.savefig('Fcl_lr_rnn_xgboost_m_roc.png')


large = 28; med = 20; small = 12
params = {'axes.titlesize': med, #子图上的标题字体大小
          'legend.fontsize': 28, #图例的字体大小
          'legend.loc': 'lower right',
          #'figure.figsize': (16, 10), #图像的画布大小
          'axes.labelsize': 28, #标签的字体大小
          'xtick.labelsize': 28, #x轴上的标尺的字体大小
          'ytick.labelsize': 28, #y轴上的标尺的字体大小
          'figure.titlesize': 28} #整个画布的标题字体大小
plt.rcParams.update(params) #设定各种各样的默认属性
# plt.style.use('seaborn-whitegrid') #设定整体风格
# sns.set_style("white") #设定整体背景风格

from sklearn.metrics import roc_curve
from sklearn import metrics

FPR_Batch, Recalll_Batch, thresholds_Batch = roc_curve(fey_valid.tolist(),feBatch_pred[:,1].tolist(),pos_label=1)


from sklearn.metrics import roc_auc_score as AUC
area_Batch = AUC(fey_valid.tolist(),feBatch_pred[:,1].tolist())


maxindex_Batch = (Recalll_Batch - FPR_Batch).tolist().index(max(Recalll_Batch - FPR_Batch))


plt.figure(figsize=(10,7), dpi= 500)
bwith = 3
TK = plt.gca()
TK.spines['bottom'].set_linewidth(bwith)
TK.spines['left'].set_linewidth(bwith)
TK.spines['top'].set_linewidth(bwith)
TK.spines['right'].set_linewidth(bwith)


plt.plot(FPR_Batch, Recalll_Batch, linestyle='-',color='red',linewidth = 6.0,label='Fully connected layer_ROC curve' % area_Batch)

plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
bwith = 3 

plt.scatter(FPR_Batch[maxindex_Batch],Recalll_Batch[maxindex_Batch],c="red",s=150)


plt.xlim([-0.02,1.02])
plt.ylim([-0.02,1.02])
font2 = {
    'family':'Times New Roman',
    'weight':'normal',
    'size':35,
    'style':'italic'
}
font1 = {
    'family':'Times New Roman',
    'weight':'normal',
    'size':20
}
plt.xlabel('1-Specificity',font2)
plt.ylabel('Sensitivily',font2)
l1 = plt.legend(loc = "lower right",fontsize = 20,prop=font1)
plt.savefig('Fcl_lr_rnn_xgboost_fe_roc.png')

print("full connect layer:")
print('precision:',metrics.precision_score(fey_valid.tolist(),feBatch_pred[:,1].detach().numpy().round()))
print('recall:',metrics.recall_score(fey_valid.tolist(),feBatch_pred[:,1].detach().numpy().round()))
print('f1:',metrics.f1_score(fey_valid.tolist(),feBatch_pred[:,1].detach().numpy().round()))
print('accuracy:',metrics.accuracy_score(fey_valid.tolist(),feBatch_pred[:,1].detach().numpy().round()))
print("auc:",area_Batch)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(fey_valid.tolist(),feBatch_pred[:,1].detach().numpy().round()).ravel()
specificity = tn / (tn+fp)
print('specificity',specificity)
print('\n')


fedata_External.drop(['local',],axis = 1,inplace = True)

fedata_ExternalX = fedata.iloc[:,fedata_External.columns != "label"]
fedata_Externaly = fedata.iloc[:,fedata_External.columns == "label"]

################################3
scaler = StandardScaler() #实例化
fedata_ExternalX_ = pd.DataFrame(scaler.fit_transform(fedata_ExternalX))
from sklearn.model_selection import train_test_split
feXtrain, feXtest, feYtrain, feYtest = train_test_split(fedata_ExternalX_,fedata_Externaly,test_size=0.1)


for i in [feXtrain, feXtest, feYtrain, feYtest]:
    i.index = range(i.shape[0])
    
feXtrain.head()
feYtrain = feYtrain.iloc[:,0].ravel()
feYtest = feYtest.iloc[:,0].ravel()
##########################################


sm = SMOTE(random_state=42) #实例化
fedata_ExternalX, fedata_Externaly = sm.fit_resample(fedata_ExternalX, fedata_Externaly)
fedata_ExternalX = fedata_ExternalX.iloc[:]
fedata_Externaly = fedata_Externaly.iloc[:,0].ravel()
fedata_ExternalX = torch.tensor(fedata_ExternalX.values, dtype = torch.float)
fedata_Externaly = torch.tensor(fedata_Externaly)
#tensor 通 array
for i in [fedata_ExternalX, fedata_Externaly]:
    i.index = range(i.shape[0])
print(fedata_ExternalX.shape, fedata_Externaly.shape)
print("------end-------")

fedata_ExternalX_Batch_pred = feBatch_model(fedata_ExternalX)

large = 28; med = 20; small = 12
params = {'axes.titlesize': med, 
          'legend.fontsize': 28, 
          'legend.loc': 'lower right',
          #'figure.figsize': (16, 10),
          'axes.labelsize': 28, 
          'xtick.labelsize': 28,
          'ytick.labelsize': 28, 
          'figure.titlesize': 28}
plt.rcParams.update(params) 

from sklearn.metrics import roc_curve
from sklearn import metrics

FPR_Batch, Recalll_Batch, thresholds_Batch = roc_curve(fedata_Externaly.tolist(),fedata_ExternalX_Batch_pred[:,1].tolist(),pos_label=1)

from sklearn.metrics import roc_auc_score as AUC
area_Batch = AUC(fedata_Externaly.tolist(),fedata_ExternalX_Batch_pred[:,1].tolist())


maxindex_Batch = (Recalll_Batch - FPR_Batch).tolist().index(max(Recalll_Batch - FPR_Batch))

plt.figure(figsize=(10,7), dpi= 500)
bwith = 3
TK = plt.gca()
TK.spines['bottom'].set_linewidth(bwith)
TK.spines['left'].set_linewidth(bwith)
TK.spines['top'].set_linewidth(bwith)
TK.spines['right'].set_linewidth(bwith)

plt.plot(FPR_Batch, Recalll_Batch, linestyle='-',color='red',linewidth = 6.0,label='Fully connected layer_ROC curve' % area_Batch)

plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
bwith = 3 


plt.scatter(FPR_Batch[maxindex_Batch],Recalll_Batch[maxindex_Batch],c="red",s=150)


plt.xlim([-0.02,1.02])
plt.ylim([-0.02,1.02])
font2 = {
    'family':'Times New Roman',
    'weight':'normal',
    'size':35,
    'style':'italic'
}
font1 = {
    'family':'Times New Roman',
    'weight':'normal',
    'size':20
}
plt.xlabel('1-Specificity',font2)
plt.ylabel('Sensitivily',font2)
l1 = plt.legend(loc = "lower right",fontsize = 20,prop=font1)
plt.savefig('Fcl_external_roc.png')
